# **LIMPIEZA DE DATOS**

In [ ]:
%matplotlib notebook
%matplotlib inline

CARGA DE LIBRERIAS

In [ ]:
import pandas as pd
import numpy as np
import os
import missingno as msno 
import matplotlib.pyplot as plt
import seaborn as sns

CARGA DE FICHEROS

In [ ]:
CARPETA_DATOS_ORIGINALES = 'Datos/Originales/'
df_sabi_1= pd.read_excel(os.path.join(CARPETA_DATOS_ORIGINALES, 'df_sabi_modif_1.xlsx'))
df_sabi_2= pd.read_excel(os.path.join(CARPETA_DATOS_ORIGINALES, 'df_sabi_modif_2_new.xlsx'))
df_dealroom= pd.read_excel(os.path.join(CARPETA_DATOS_ORIGINALES, 'df_dealroom_modif.xlsx'))
df_sabi_3= pd.read_excel(os.path.join(CARPETA_DATOS_ORIGINALES, 'df_sabi_parte3.xlsx'))

VISUALIZACIi“N DE FICHEROS

In [ ]:
df_sabi_1.info()

In [ ]:
df_sabi_2.info()

In [ ]:
df_dealroom.info()

In [ ]:
print(df_sabi_1.isna().sum())
print(df_sabi_1.shape)

In [ ]:
df_sabi_2.columns

In [ ]:
print(df_sabi_2.isna().sum())
print(msno.matrix(df_sabi_2))

In [ ]:
print(df_sabi_3.isna().sum())
print(msno.matrix(df_sabi_3))

CONVERSION DE VALORES Y CREACION DE VARIABLES

###### df_sabi2

In [ ]:
# pasamos el n.s. a NaN con applymap
df_sabi_2=df_sabi_2.applymap(lambda x: np.nan if x=='n.s.' else x)
df_sabi_3=df_sabi_3.applymap(lambda x: np.nan if x=='n.s.' else x)

In [ ]:
print(df_sabi_2.shape)
# se calcula la cantidad de nulos por fila
# print(df_sabi_2.isna().sum(axis=1)[df_sabi_2.isna().sum(axis=1)>8])
indices_missings= df_sabi_2.isna().sum(axis=1)[df_sabi_2.isna().sum(axis=1)>8].index
print(df_sabi_2.iloc[indices_missings,:])

In [ ]:
# se crea columna con numero de missing por fila
df_sabi_2['n_missings']= df_sabi_2.isna().sum(axis=1)

In [ ]:
#Se realizan boxplots de las columnas numericas que esti©n mostradas en %.
df_sabi_2_porcentaje = df_sabi_2.loc[:, df_sabi_2.columns.str.contains('%')]
fig, ax = plt.subplots(figsize=(20, 8))
sns.boxplot(data=df_sabi_2_porcentaje, ax=ax)
plt.xticks(rotation=90)
ax.set_ylim(-500, 500)
ax.yaxis.set_major_locator(plt.MultipleLocator(100))
ax.set_xticklabels(ax.get_xticklabels(), fontsize=12)
plt.show()

###### df_sabi_3

In [ ]:
# de df_sabi_3 nos quedamos solo con 4 variables
df_sabi_3_final= df_sabi_3[['Codigo_NIF','year', 'Gastos de personal mil EUR', 'Coste medio de los empleados mil']]

VISUALIZACION DE VALORES AUSENTES

In [ ]:
print(df_dealroom.isna().sum())

In [ ]:
df_dealroom.describe()

In [ ]:
# se pasa el growth stage a dummy
mapping = {'seed': 0, 'early growth': 1, 'late growth': 2}
df_dealroom['growth_stage'] = df_dealroom['growth_stage'].map(mapping)

In [ ]:
# se sustituyen las letras con tildes por las letras sin tildes de todas las columnas de df_sabi_2
df_sabi_2.columns= df_sabi_2.columns.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
# lo mismo con el resto de dataframes
df_sabi_1.columns= df_sabi_1.columns.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
df_dealroom.columns= df_dealroom.columns.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
df_sabi_3_final.columns= df_sabi_3_final.columns.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [ ]:
# correlograma de missing values
msno.heatmap(df_dealroom)

In [ ]:
# columnas financieras
columnas_financieras= df_sabi_2.columns 
columnas_financieras= columnas_financieras.append(df_sabi_3_final.columns)
columnas_financieras.shape

In [ ]:
df_dealroom[df_dealroom['n_empleados_dealroom'].isna()]

CONVERSION DE VARIABLES

In [ ]:
# se convierte la primera letra de cada palabra a mayuscula
df_dealroom['last_funding_date']= df_dealroom['last_funding_date'].str.capitalize()
df_dealroom['first_funding_date']= df_dealroom['first_funding_date'].str.capitalize()
# hay un valor de fecha que aparece solo el ai±o, se pasa a formato mes/ai±o
indice = df_dealroom[df_dealroom['last_funding_date']=='2021'].index[0]
df_dealroom.at[indice, 'last_funding_date'] = 'Jan/2021'
indice = df_dealroom[df_dealroom['first_funding_date']=='2003'].index[0]
df_dealroom.at[indice, 'first_funding_date'] = 'Jan/2003'
indice = df_dealroom[df_dealroom['first_funding_date']=='2017'].index[0]
df_dealroom.at[indice, 'first_funding_date'] = 'Jan/2017'
# se pasan las fechas a datetime
df_dealroom['last_funding_date']= pd.to_datetime(df_dealroom['last_funding_date'], format='%b/%Y')
df_dealroom['first_funding_date']= pd.to_datetime(df_dealroom['first_funding_date'], format='%b/%Y')


In [ ]:
print(df_dealroom['b2b_b2c'].value_counts())
print(df_dealroom['b2b_b2c'].isna().sum())

In [ ]:
# se cogen las empresas que el campo b2b_b2c es NaN
empresas_b2b_b2c_nan= df_dealroom[df_dealroom['b2b_b2c'].isna()]
empresas_b2b_b2c_nan

In [ ]:
# se busca si es b2b o b2c a mano y se crea una funcion para automatizarlo
lista_empresa_consumer= ['Feelfree Rentals', 'Modfie', 'Worldpats', 'Kimet Sport', 'Puntodis', 'DNA Data' ]
lista_empresa_bussines= ['Hub Gasteiz', 'Quevedos Strategic Partners', 'Anbiolab', 'Solid Machine Vision', 'Innovative Hall Media Technologies', 'lorke systems',
'Gistek Insurance Solutions', 'VIRTUALWARE', 'Naivan', ]
for empresa in lista_empresa_consumer:
    df_dealroom.iloc[df_dealroom[df_dealroom['name_dealroom'] == empresa].index[0],13]= 'consumer'
for empresa in lista_empresa_bussines:
    df_dealroom.iloc[df_dealroom[df_dealroom['name_dealroom'] == empresa].index[0],13]= 'business'


In [ ]:
# se pasan b2b a 1 en una columna, b2c a 1 en otra, y se dejan los que son ambos a 0
df_dealroom['b2b']= df_dealroom['b2b_b2c'].apply(lambda x: 1 if 'business' in x else 0)
df_dealroom['b2c']= df_dealroom['b2b_b2c'].apply(lambda x: 1 if 'consumer' in x else 0)

UNIi“N DE LOS DATAFRAMES EN UNO išNICO

In [ ]:
# juntamos los dos df_sabi
df_sabi= pd.merge(df_sabi_1, df_sabi_2, on='Codigo_NIF', how='inner')
df_sabi_completo= pd.merge(df_sabi, df_sabi_3_final, on=['Codigo_NIF','year'], how='left')
# juntamos este df con el de dealroom
df= pd.merge(df_sabi_completo, df_dealroom, on='Codigo_NIF', how='inner')
df

In [ ]:
print(msno.matrix(df))

In [ ]:
# se visualiza de las columnas en las que hay missings en la columna de empleados normal, cuantos missings hay en la de dealroom
print(df['Numero empleados'].isna().sum())
print(df[df.loc[:,'Numero empleados'].isna()]['n_empleados_dealroom'].isna().sum())
# hay 10 valores que no hay en la columna de sabi, pero si en la de dealroom, por lo que se indica
df['Numero empleados']= df.loc[:,'Numero empleados'].fillna(df['n_empleados_dealroom'])
# Ahora solo quedan 70 valores nulos en la columna de empleados, que son los que no tienen en ninguno de las dos df

In [ ]:
# se quita la columna de n_empleados_dealroom
df= df.drop('n_empleados_dealroom', axis=1)

In [ ]:
summary_inicial= df.describe()

# Ideas de creacion de variables

In [ ]:
# Tiempo que lleva la empresa en el mercado
# Tiempo desde creacion hasta primer round
# Tiempo desde primer round hasta ultimo round

CREACION DE NUEVAS VARIABLES EN EL NUEVO DATAFRAME

In [ ]:
df['Fecha constitucion']=pd.to_datetime(df['Fecha constitucion'], format='%Y/%m/%d')

In [ ]:
#se crea una variable para conocer los ai±os que lleva la empresa en el mercado
df['Anos en Mercado']= (2023-df['Fecha constitucion'].dt.year)
columnas = list(df.columns)
columnas.remove('Anos en Mercado')

# se altera el orden de las columnas para que los ai±os en el mercado vayan despues de la fecha de constitucion
columnas = columnas[:columnas.index("Fecha constitucion")+1] + ['Anos en Mercado'] + columnas[columnas.index("Fecha constitucion")+1:]
df = df.reindex(columns=columnas)

In [ ]:
#Boxplot de la valoraciones en 2022.
sns.boxplot(y=df['Anos en Mercado'])
plt.xlabel('Anos en Mercado', weight='bold')
plt.ylabel('', weight='bold')
plt.show()

In [ ]:
# revenue models a dummy con sklearn e ignorando los missings
df['revenue_models']= df['revenue_models'].fillna('missing')
opciones = ['saas', 'missing' ,'manufacturing', 'marketplace & ecommerce']
for opcion in opciones:
    df[f'revenue_{opcion}']= df['revenue_models'].apply(lambda x: 1 if opcion in x else 0)

In [ ]:
df['last_funding_date']=pd.to_datetime(df['last_funding_date'], format='%Y/%m/%d')

In [ ]:
#se crea una variable para conocer los ai±os desde que se realizon la ultima financiacion.
df['Anos desde ultima financiacion']= (2023-df['last_funding_date'].dt.year)
df['Anos desde ultima financiacion'].fillna(0, inplace=True)
df['Anos desde ultima financiacion'] = df['Anos desde ultima financiacion'].astype(int)

# se altera el orden de las columnas para que los ai±os desde ultima financiacion vayan despues de la fecha de ultima financiacion.
columnas = list(df.columns)
columnas.remove('Anos desde ultima financiacion')
columnas = columnas[:columnas.index('last_funding_date')+1] + ['Anos desde ultima financiacion'] + columnas[columnas.index('last_funding_date')+1:]
df = df.reindex(columns=columnas)

In [ ]:
# se ven cuantas filas tienen missing la columna de last_funding entre las que valuation_2022 no es missing
print(df[(df['last_funding'].isna()) & (df['valuation_2022'].notna())].shape)
print(df[df['valuation_2022'].notna()].shape)
# una de cada 4 empresas que tiene valoracion en 2022 tiene missing en last_funding, asi que estari­a bien imputarla

In [ ]:
#Boxplot de la valoraciones en 2022.
sns.boxplot(y=df['valuation_2022'])
plt.xlabel('Valoracion 2022', weight='bold')
plt.ylabel('', weight='bold')
plt.show()

# Otras cosas a hacer

In [ ]:
df_finanzas= df.copy()

In [ ]:
# Quitar columnas con muchos missings (ejemplo: deuda)


In [ ]:
df['n_missings']=df.isna().sum(axis=1)
x=pd.DataFrame(df['n_missings'])

In [ ]:
df['Inmovilizado mil EUR']=df['Inmovilizado mil EUR'].fillna(df['Total activo mil EUR'] - df['Activo circulante mil EUR'])
df['Activo circulante mil EUR']=df['Activo circulante mil EUR'].fillna(df['Total activo mil EUR'] - df['Inmovilizado mil EUR'].fillna(0))
df['Total activo mil EUR']=df['Total activo mil EUR'].fillna(df['Total pasivo y capital propio mil EUR'])

In [ ]:
print((df['Activo circulante mil EUR']==df['Activo circulante mil EUR'].fillna(df['Total activo mil EUR'] - df['Inmovilizado mil EUR'])).sum())
print((df['Inmovilizado mil EUR']==df['Inmovilizado mil EUR'].fillna(df['Total activo mil EUR'] - df['Activo circulante mil EUR'])).sum())

In [ ]:
#Sabiendo que -> df['Pasivo fijo mil EUR']=df['Pasivo fijo mil EUR'].fillna(df['Acreedores a L. P. mil EUR'].fillna(0)+df['Deudas financieras mil EUR'])
a= df['Pasivo liquido mil EUR']== df['Deudas financieras mil EUR'].fillna(0)+df['Acreedores comerciales mil EUR'].fillna(0)
print(a.sum())
b= df['Activo circulante mil EUR']== df['Existencias mil EUR'].fillna(0)+df['Deudores mil EUR'].fillna(0)+ df['Tesoreria mil EUR'].fillna(0)
print(b.sum())
print(df.shape)

In [ ]:
(df['Pasivo fijo mil EUR']-df['Acreedores comerciales mil EUR'] == df['Deudas financieras mil EUR']).sum()

In [ ]:
# df['Fondos propios mil EUR']=df['Fondos propios mil EUR'].fillna(df['Capital suscrito mil EUR'].fillna(0)+df['Otros fondos propios mil EUR'].fillna(0))
df['Pasivo fijo mil EUR']=df['Pasivo fijo mil EUR'].fillna(df['Total pasivo y capital propio mil EUR'].fillna(0)-df['Pasivo liquido mil EUR'].fillna(0)-df['Fondos propios mil EUR'].fillna(0))
df['Pasivo liquido mil EUR']=df['Pasivo liquido mil EUR'].fillna(df['Total pasivo y capital propio mil EUR'].fillna(0)-df['Pasivo fijo mil EUR'].fillna(0)-df['Fondos propios mil EUR'].fillna(0))
df['Total pasivo']=(df['Pasivo fijo mil EUR']+df['Pasivo liquido mil EUR'])


In [ ]:
# imputamos numero de empleados con gastos de personal y coste medio por empleado
df['Numero empleados']=df['Numero empleados'].fillna(df['Gastos de personal mil EUR']/df['Coste medio de los empleados mil'])
df['Coste medio de los empleados mil']=df['Coste medio de los empleados mil'].fillna(df['Gastos de personal mil EUR']/df['Numero empleados'])
df['Gastos de personal mil EUR']=df['Gastos de personal mil EUR'].fillna(df['Numero empleados']*df['Coste medio de los empleados mil'])


In [ ]:
# imputamos el coste medio de los empleados haciendo una media por sector


In [ ]:
(df['Resultado Actividades Ordinarias mil EUR']==df['Result. ordinarios antes Impuestos mil EUR']- df['Impuestos sobre sociedades mil EUR'].fillna(0)).sum()

In [ ]:
#Realiza estos calculos si alguna de estas columnas es nula, y sustituye el valor nulo por el calculado
df['Impuestos sobre sociedades mil EUR']=df['Impuestos sobre sociedades mil EUR'].fillna(df['Result. ordinarios antes Impuestos mil EUR']-df['Resultado Actividades Ordinarias mil EUR'])

In [ ]:
localidad= df['Localidad']

In [ ]:
# Quitar columna forma juridica o forma juridica detallada, lo mismo con estado y estado detallado
df= df.drop('Forma juridica detallada', axis=1)
df= df.drop('Estado detallado', axis=1)
# se quita la columna de n_empleados_dealroom
df= df.drop('n_empleados_dealroom', axis=1)
# Quitar columnas que no aportan informacion (ejemplo: nombre de la empresa, pagina web, ciudad, etc)
df= df.drop('Localidad', axis=1)
# quitamos la columna de codigo de consolidacion porque casi todos los valores son de un tipo
df= df.drop('Codigo consolidacion', axis=1)
# quitamos la columna de estado porque son todos activas
df= df.drop('Estado', axis=1)
# quitamos tagline 
df= df.drop('tagline', axis=1)
# tambien website
df= df.drop('website', axis=1)
# profile_url
df= df.drop('profile_url', axis=1)
# name_dealroom porque ya salen en sabi
df= df.drop('name_dealroom', axis=1)
# n_missings
df= df.drop('n_missings', axis=1)
# forma juridica
df= df.drop('Forma juridica', axis=1)
# company status
df= df.drop('company_status', axis=1) # el 95% son operational
# b2bc
df= df.drop('b2b_b2c', axis=1) # ya hay columnas si es b2b o b2c
# revenue models
df= df.drop('revenue_models', axis=1) # ya hay columnas dummies
# quitamos las fechas
df= df.drop('last_funding_date', axis=1)
df= df.drop('first_funding_date', axis=1)
df= df.drop('Fecha constitucion', axis=1)
# resultado de actividades ordinarias porque es igual que resultado del ejercicio
df= df.drop('Resultado Actividades Ordinarias mil EUR', axis=1)
# quitamos los inmovilizados materiales e inmateriales y nos quedamos con inmovilizado
df= df.drop('Inmovilizado material mil EUR', axis=1)
df= df.drop('Inmovilizado inmaterial mil EUR', axis=1)
# eliminamos existencias y â‚¬ % existencias
df= df.drop('Existencias mil EUR', axis=1)
df= df.drop('Rotacion de las existencias %', axis=1)
# quitamos dotaciones para amortizaciones de inmovilizado
df= df.drop('Dotaciones para amortiz. de inmovil. mil EUR', axis=1)
# gastos financieros
df= df.drop('Gastos financieros y gastos asimilados mil EUR', axis=1)
# free capital por ser todo ceros
df= df.drop('Free capital mil EUR', axis=1)
# otros fondos propios
df= df.drop('Otros fondos propios mil EUR', axis=1)

In [ ]:
df['n_missings']=df.isna().sum(axis=1)
x['df_missings_final']= df['n_missings']

In [ ]:
# buscamos los total_funding que son 0
print(df[df['total_funding']==0].shape)
# cuando el total_funding es 0, el last_funding tambii©n lo imputamos como 0
df.loc[df['total_funding']==0, 'last_funding']=0

# creamos ratio de last_funding sobre total_funding
df['ratio_last_funding']=df['last_funding']/df['total_funding']

In [ ]:
print(df['total_funding'].describe())
print(df['total_funding'].isna().sum())
print(df['ratio_last_funding'].describe())

In [ ]:
# hacer el calculo de la diferencia entre las variables en el 2020 y 2021 (asi­ se puede sustituir por el valor del 2020)

CREACION DE RATIOS EN EL DF completo

In [ ]:
# precio/ventas
# precio/ebitda
# precio/ebit
# precio/benecios (si tienen)
# tasa crecimiento ventas
# tasa crecimiento ingresos
# tasa crecimiento beneficios
# ratio de liquidez
# ratio de endeudamiento
# margen bruto
# Cash Burn Rate

In [ ]:
df.loc[df['Pasivo fijo mil EUR']<0, 'Pasivo fijo mil EUR']=0
df.loc[df['Pasivo liquido mil EUR']<0, 'Pasivo liquido mil EUR']=0

In [ ]:
df.loc[:,'Ratio de liquidez %']=df['Ratio de liquidez %'].fillna(((df['Activo circulante mil EUR'])/(df['Pasivo liquido mil EUR'])))
df.loc[:,'Ratio de solvencia %']=df['Ratio de solvencia %'].fillna(df['Ratio de liquidez %'])
df.loc[:,'Ratio_endeudamiento']=df['Total pasivo y capital propio mil EUR']/df['Total pasivo']

df.loc[:,'Margen_bruto(costes trabajadores)']=(df['Ingresos de explotacion mil EUR']-df['Coste medio de los empleados mil'])/(df['Ingresos de explotacion mil EUR']*100)

In [ ]:
# se ven cuantas columnas tienen 1 o mas missings
print(df.isna().sum()[df.isna().sum()>0].sort_values(ascending=False).count())
variables_con_missings= df.isna().sum()[df.isna().sum()>0].sort_values(ascending=False)
variables_con_missings_columnas= list(variables_con_missings.index)

Se continua con la imputacion de variables antes de crear nuevos df

In [ ]:
# se actualiza la lista de variables financieras
variables_financieras_eliminadas= ['Resultado Actividades Ordinarias mil EUR', 'Inmovilizado material mil EUR',
 'Inmovilizado inmaterial mil EUR', 'Existencias mil EUR', 'Rotacion de las existencias %', 
 'Dotaciones para amortiz. de inmovil. mil EUR', 'Gastos financieros y gastos asimilados mil EUR', 'Tesoreria mil EUR',
  'Otros fondos propios mil EUR']
columnas_financieras_completas= columnas_financieras.drop(['year', 'Codigo_NIF', 'n_missings']).drop(variables_financieras_eliminadas)
columnas_financieras_completas= columnas_financieras_completas.append(pd.Index(['Total pasivo']))

# se divide el valor de la columna en 2021 entre el valor de la columna en 2020
# y se calcula la media de crecimiento de cada empresa entre 2020 y 2021 para cada columna financiera y se 
# guarda en un diccionario y si hay un mi


# crecimiento_2020_2021 = {}
# for columna in columnas_financieras_completas:
#     diff = df.loc[df['year'] == 2021, columna].fillna(0) - df.loc[df['year'] == 2020, columna].fillna(0)
#     crecimiento_2020_2021[columna] = np.median(diff)
# crecimiento_2020_2021

In [ ]:
df_antes_nif= df.copy()

In [ ]:
# se crea un for para que si una empresa tiene un missing en una columna financiera en un año, 
# se sustituye por el del año anterior o posterior

for nif in df['Codigo_NIF'].unique():
    for columna in variables_con_missings_columnas:
        if df.loc[(df['Codigo_NIF']==nif) & (df['year']==2020), columna].isna().values[0]:
            df.loc[(df['Codigo_NIF']==nif) & (df['year']==2020), columna]=df.loc[(df['Codigo_NIF']==nif) & (df['year']==2021), columna].values[0]
        if df.loc[(df['Codigo_NIF']==nif) & (df['year']==2021), columna].isna().values[0]:
            df.loc[(df['Codigo_NIF']==nif) & (df['year']==2021), columna]=df.loc[(df['Codigo_NIF']==nif) & (df['year']==2020), columna].values[0]


In [ ]:
variables_eliminar= ['ratio_last_funding','last_round', 'last_funding', 'ownerships', 'Tesoreria mil EUR', 'n_missings' ]
df= df.drop(variables_eliminar, axis=1)

In [ ]:
# cuantas empresas tienen las 2 variables missings
print((df[(df['Costes de los trabajadores / Ingresos de explotacion (%) %'].isna())&(df['Ingresos de explotacion mil EUR'].isna())]).shape)
print((df[(df['Costes de los trabajadores / Ingresos de explotacion (%) %'].isna())&(df['Numero empleados'].isna())]).shape)
print((df[(df['Ingresos de explotacion mil EUR'].isna())&(df['Importe neto Cifra de Ventas mil EUR'].isna())]).shape)

In [ ]:
df.loc[df['growth_stage'].isna(), 'growth_stage']= 0 # se asigna el valor más común
df.loc[df['Deudores mil EUR'].isna(), 'Deudores mil EUR']= 0 # se considera que no hay deudores

In [ ]:
# correlacion deudas financieras con otras variables
df.corr()['Deudas financieras mil EUR'].sort_values(ascending=False)

In [ ]:
# para imputar coste medio de los empleados, se agrupa por CNAE y se calcula el coste medio
df['Codigo primario CNAE adaptado']= df['Codigo primario CNAE 2009'].apply(lambda x: round(x/100,0))
df['Coste medio de los empleados mil']=df['Coste medio de los empleados mil'].fillna(df.groupby('Codigo primario CNAE adaptado')['Coste medio de los empleados mil'].transform('median'))
df['Costes de los trabajadores / Ingresos de explotacion (%) %']= df['Costes de los trabajadores / Ingresos de explotacion (%) %'].fillna(df.groupby('Codigo primario CNAE 2009')['Costes de los trabajadores / Ingresos de explotacion (%) %'].transform('median'))
df['Deudas financieras mil EUR']= df['Deudas financieras mil EUR'].fillna(df.groupby('Codigo primario CNAE 2009')['Deudas financieras mil EUR'].transform('median'))
df['Acreedores a L. P. mil EUR']= df['Acreedores a L. P. mil EUR'].fillna(df.groupby('Codigo primario CNAE 2009')['Acreedores a L. P. mil EUR'].transform('median'))
df['Acreedores comerciales mil EUR']= df['Acreedores comerciales mil EUR'].fillna(df.groupby('Codigo primario CNAE 2009')['Acreedores comerciales mil EUR'].transform('median'))
df['Periodo de cobro (dias) dias']= df['Periodo de cobro (dias) dias'].fillna(df.groupby('Codigo primario CNAE 2009')['Periodo de cobro (dias) dias'].transform('median'))
df['Margen de beneficio (%) %']= df['Margen de beneficio (%) %'].fillna(df.groupby('Codigo primario CNAE 2009')['Margen de beneficio (%) %'].transform('median'))
df= df.drop('Codigo primario CNAE adaptado', axis=1)

In [ ]:
from sklearn.linear_model import LinearRegression

def imputacion_reg_lineal(df, var_dependiente, var_independiente):
    missings = df[df[var_dependiente].isna()]
    no_missings = df.dropna(subset=[var_dependiente])

    modelo = LinearRegression().fit(no_missings[[var_independiente]], no_missings[var_dependiente])
    prediccion = modelo.predict(missings[[var_independiente]])
    df.loc[df[var_dependiente].isna(), var_dependiente] = prediccion

    return df

In [ ]:
# se busca primero la correlacion entre las variables e ingresos de explotacion
df.corr()['Ingresos de explotacion mil EUR'].sort_values(ascending=False) # deudores tiene una correlacion muy alta
# se imputan los ingresos de explotacion
imputacion_reg_lineal(df, 'Ingresos de explotacion mil EUR', 'Deudores mil EUR')

In [ ]:
df['Coste medio de los empleados mil']=df['Coste medio de los empleados mil'].fillna(df['Costes de los trabajadores / Ingresos de explotacion (%) %']* df['Ingresos de explotacion mil EUR']/100/df['Numero empleados'])

In [ ]:
# numero de empleados
df['Numero empleados']=df['Numero empleados'].fillna(round((df['Costes de los trabajadores / Ingresos de explotacion (%) %']* df['Ingresos de explotacion mil EUR']/100/df['Coste medio de los empleados mil']),0))
df['Gastos de personal mil EUR']= df['Gastos de personal mil EUR'].fillna(df['Coste medio de los empleados mil']*df['Numero empleados'])

In [ ]:
# se crea ahora que no hay NAs
df.loc[:,'Margen_bruto(costes trabajadores)']=(df['Ingresos de explotacion mil EUR']-df['Coste medio de los empleados mil'])/(df['Ingresos de explotacion mil EUR']*100)

In [ ]:
# cor de total_funding
df.corr()['total_funding'].sort_values(ascending=False) # se ve que hay una correlacion con el numero de empleados

In [ ]:
# importe neto de ventas con ingresos de explotacion
imputacion_reg_lineal(df, 'Importe neto Cifra de Ventas mil EUR', 'Ingresos de explotacion mil EUR')
# total funding con gastos financieros
imputacion_reg_lineal(df, 'Deudas financieras mil EUR', 'Total pasivo')
imputacion_reg_lineal(df, 'total_funding', 'Gastos financieros mil EUR')
imputacion_reg_lineal(df, 'Acreedores comerciales mil EUR', 'Deudas financieras mil EUR')
imputacion_reg_lineal(df, 'Acreedores a L. P. mil EUR', 'Gastos financieros mil EUR')

# hay 3 valores de importe neto de ventas que son menores que 0, así que se ponen en positivo
df.loc[df['Importe neto Cifra de Ventas mil EUR']<0, 'Importe neto Cifra de Ventas mil EUR']= \
df.loc[df['Importe neto Cifra de Ventas mil EUR']<0, 'Importe neto Cifra de Ventas mil EUR']*-1

In [ ]:
print(df['Deudas financieras mil EUR'].describe())
print(df[df['Deudas financieras mil EUR']>=0]['Deudas financieras mil EUR'].describe())
aa= df[df['Deudas financieras mil EUR']<0]

In [ ]:
df.isna().sum()[df.isna().sum()>0].sort_values(ascending=False)

In [ ]:
# se buscan empresas que no coincida la suma de pasivos y fondos propios con el total de activos
pasivo_empresas=df[round((df['Total pasivo']+ df['Fondos propios mil EUR']),5)!= round(df['Total activo mil EUR'],5)]
df.loc[pasivo_empresas.index, 'Pasivo fijo mil EUR']= (df.loc[pasivo_empresas.index, 'Total activo mil EUR'] - df.loc[pasivo_empresas.index, 'Fondos propios mil EUR']) * 0.47
df.loc[pasivo_empresas.index, 'Pasivo liquido mil EUR']= (df.loc[pasivo_empresas.index, 'Total activo mil EUR'] - df.loc[pasivo_empresas.index, 'Fondos propios mil EUR']) * 0.53

In [ ]:
df.loc[df['Numero empleados']<=0, 'Numero empleados']=1

In [ ]:
# variables con outliers
sumary= df.describe()

In [ ]:
df[df['Fondos propios mil EUR']<0]

In [ ]:
df=df.applymap(lambda x: 99999 if x== np.inf else x)
df=df.applymap(lambda x: -99999 if x== -np.inf else x)

Creacion dfs nuevos

In [ ]:
df_pivotado= df.pivot_table(index='Codigo_NIF', columns='year', values=columnas_financieras_completas,)
df_pivotado_columnas= df_pivotado.columns
df_pivotado_copia= df_pivotado.copy()

In [ ]:
# se pasan las columnas de los años de df_pivotado como sufijo de las columnas de df_pivotado
df_pivotado.columns= df_pivotado.columns.map('{0[0]}_{0[1]}'.format)
df_pivotado= df_pivotado.reset_index()
# luego se añaden el resto de columnas de df seleccionando las que no estan en df_pivotado
columnas= columnas_financieras_completas.tolist()
df_adquisicion_completo= pd.merge(df_pivotado, df.loc[:,~df.columns.isin(columnas)], on= 'Codigo_NIF', how='left')

# se quita 1 de cada 2 filas porque hay duplicados
df_adquisicion_completo= df_adquisicion_completo.drop_duplicates(subset='Codigo_NIF', keep='first')

df_adquisicion_completo

In [ ]:
# se crea otro df quitando las columnas de 2020
df_adquisicion_2021= df_adquisicion_completo.drop(df_adquisicion_completo.filter(regex='_2020').columns, axis=1)
df_adquisicion_2021

In [ ]:
# se calcula el ratio de crecimiento de las variables financieras entre 2020 y 2021
df_pivotado_ratios= pd.DataFrame()
for col in df_pivotado_columnas.get_level_values(0):
    df_pivotado_ratios[col+'_ratio']=df_pivotado_copia[col][2021]/df_pivotado_copia[col][2020]
df_pivotado_ratios= df_pivotado_ratios.reset_index()

In [ ]:
# se junta el df de ratios con el df de adquisicion
df_adquisicion_final= pd.merge(df_pivotado_ratios,df_adquisicion_2021 , on='Codigo_NIF', how='left')
df_adquisicion_final=df_adquisicion_final.applymap(lambda x: 99999 if x== np.inf else x)
df_adquisicion_final=df_adquisicion_final.applymap(lambda x: -99999 if x== -np.inf else x)
df_adquisicion_final= df_adquisicion_final.fillna(0)
df_adquisicion_final

In [ ]:
df_graficos= df_adquisicion_final.copy()
df_graficos['Localidad']= localidad

CREACION DEL DATAFRAME **DF Valoracion**

In [ ]:
# Primero se crea el df que esti© preparado para hacer el modelo de valoracion de empresas
# se busca cuando valoracion no es na
df_valoracion= df_adquisicion_final[df_adquisicion_final['valuation_2022']!=0]
df_valoracion.shape

In [ ]:
# se quita la columna de valuation
df_adquisicion_final= df_adquisicion_final.drop('valuation_2022', axis=1)

In [ ]:
df_valoracion['Nombre_sabi'].unique().shape # 60 empresas con valoracion
# se quita la segunda instancia de cada empresa y se queda con la del 2021
df_valoracion= df_valoracion.drop_duplicates(subset='Codigo_NIF', keep='first')
df_valoracion= df_valoracion.drop('year', axis=1)
df_valoracion.shape

In [ ]:
#Realiza estos calculos si alguna de estas columnas es nula, y sustituye el valor nulo por el calculado
df_valoracion.loc[:,'Precio/Venta']=(df_valoracion['valuation_2022']/df_valoracion['Importe neto Cifra de Ventas mil EUR_2021'])
df_valoracion.loc[:,'Precio/Ebitda']=(df_valoracion['valuation_2022']/df_valoracion['EBITDA mil EUR_2021'])
df_valoracion.loc[:,'Precio/Ebit']=(df_valoracion['valuation_2022']/df_valoracion['EBIT mil EUR_2021'])

#Se reemplazan valores infinitos con valores altos.
df_valoracion.loc[:,'Precio/Venta']=df_valoracion['Precio/Venta'].replace([np.inf, -np.inf], [99999, -99999])

In [ ]:
#Se realizan boxplots de las columnas numericas que esten mostradas en %.
df_porcentaje = df_valoracion.loc[:, df_valoracion.columns.str.contains('%_2021')]
fig, ax = plt.subplots(figsize=(20, 8))
sns.boxplot(data=df_porcentaje, ax=ax)
plt.xticks(rotation=90)
ax.set_ylim(-500, 500)
ax.yaxis.set_major_locator(plt.MultipleLocator(100))
ax.set_xticklabels(ax.get_xticklabels(), fontsize=12)
plt.show()

In [ ]:
# se saca el codigo_nif de las empresas que tienen mas de 20 missings porque son empresas que no tienen datos para el 2020
# y no se pueden calcular los ratios con ellos

# empresas_missings= df[df['n_missings']>20]['Codigo_NIF'].unique()
# df= df[~df['Codigo_NIF'].isin(empresas_missings)]
# print(df.shape)

Seleccion de variables para el modelo de valoracion.

In [ ]:
#Calculamos la correlación de la columna 'valuation_2022' con todas las demás columnas
correlacion = df_valoracion.corr()['valuation_2022']
df_valoracion_correlacion = pd.DataFrame({'Variables': correlacion.index, 'Correlacion con valuation_2022': correlacion.values})
df_valoracion_correlacion = df_valoracion_correlacion.sort_values(by='Correlacion con valuation_2022',ascending=False)
print(df_valoracion_correlacion)

In [ ]:
#Calculamos la correlación de la columna 'valuation_2022' con todas las demás columnas, pero en valor absoluto.
correlacion = df_valoracion.corr()['valuation_2022']
df_valoracion_correlacion_valor_absoluto = pd.DataFrame({'Variables': correlacion.index, 'Correlacion con valuation_2022': correlacion.abs().values})
df_valoracion_correlacion_valor_absoluto = df_valoracion_correlacion_valor_absoluto.sort_values(by='Correlacion con valuation_2022', ascending=False)
print(df_valoracion_correlacion_valor_absoluto)

In [ ]:
# se crea la carpeta de datos limpios
CARPETA_DATOS_LIMPIOS = 'Datos/Limpios/'
if not os.path.exists(CARPETA_DATOS_LIMPIOS):
    os.makedirs(CARPETA_DATOS_LIMPIOS)

# se guarda el df de adquisicion
df_adquisicion_final.to_csv(CARPETA_DATOS_LIMPIOS + 'df_adquisicion_final.csv', index=False)

# se guarda el df de valoracion
df_valoracion.to_csv(CARPETA_DATOS_LIMPIOS + 'df_valoracion.csv', index=False)

# se guarda el df de graficos
df_graficos.to_csv(CARPETA_DATOS_LIMPIOS + 'df_graficos.csv', index=False)

In [ ]:
CARPETA_DATOS_ORIGINALES = 'Datos/Limpios/'
df_adquisicion_final= pd.read_csv(os.path.join(CARPETA_DATOS_ORIGINALES, 'df_adquisicion_final.csv'))
df_valoracion= pd.read_csv(os.path.join(CARPETA_DATOS_ORIGINALES, 'df_valoracion.csv'))

Seleccion de variables

In [ ]:
df_valoracion.shape

In [ ]:
df_valoracion.columns

In [ ]:
#Se eliminan las variables identificadoras
df_valoracion=df_valoracion.drop(["Codigo_NIF",'Nombre_sabi'],axis=1)

In [ ]:
df_valoracion_correlacion_valor_absoluto

In [ ]:
#Se eliminan las variables que tienen una correlación con 'valuation_2022' menor a 0.3
columnas_a_mantener=list(df_valoracion_correlacion_valor_absoluto[df_valoracion_correlacion_valor_absoluto\
    ['Correlacion con valuation_2022']>0.3]['Variables'].values)
columnas_a_mantener

In [ ]:
df_valoracion_seleccionado=df_valoracion[columnas_a_mantener]

In [ ]:
df_valoracion_seleccionado=df_valoracion_seleccionado.drop(['valuation_2022'],axis=1)

In [ ]:
import random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Seleccionar una lista aleatoria de características para utilizar en el modelo
num_features = 10
all_features = list(df_valoracion_seleccionado)
random.shuffle(all_features)
selected_features = all_features[:num_features]

# Dividir el conjunto de datos en características y variable objetivo
X = df_valoracion[selected_features]
y = df_valoracion['valuation_2022']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# Entrenar un modelo de regresión lineal utilizando solo las características seleccionadas
model = LinearRegression()
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Evaluar el rendimiento del modelo utilizando el coeficiente de determinación (R^2)
r2 = r2_score(y_test, y_pred)
print(f'Características utilizadas: {selected_features}, R^2: {r2}')

In [ ]:
from sklearn.datasets import load_boston
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Dividir el conjunto de datos en características y variable objetivo
X, y = df_valoracion_seleccionado, df_valoracion['valuation_2022']

# Iterar a través de diferentes valores de K y seleccionar las mejores características
for k in [5, 10, 15]:
    # Seleccionar las K mejores características
    selector = SelectKBest(score_func=f_regression, k=k)
    selector.fit(X, y)

    # Obtener las características seleccionadas
    X_selected = selector.transform(X)

    # Dividir los datos en conjuntos de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=123)

    # Entrenar un modelo de regresión lineal utilizando solo las características seleccionadas
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Realizar predicciones en el conjunto de prueba
    y_pred = model.predict(X_test)

    # Evaluar el rendimiento del modelo utilizando el coeficiente de determinación (R^2)
    r2 = r2_score(y_test, y_pred)
    print(f'Para k={k}, R^2: {r2}')

In [ ]:
# import itertools
# from sklearn.feature_selection import SelectKBest, f_regression
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LinearRegression
# from sklearn.metrics import r2_score

# # Dividir el conjunto de datos en características y variable objetivo
# X, y = df_valoracion_seleccionado, df_valoracion['valuation_2022']

# # Crear una lista de todas las combinaciones posibles de características
# combinations = []
# for k in range(1, len(X.columns) + 1):
#     combinations += itertools.combinations(X.columns, k)

# # Variables para almacenar la mejor combinación de características y su rendimiento
# best_features = None
# best_r2 = 0

# # Iterar a través de todas las combinaciones posibles de características
# for combination in combinations:
#     # Seleccionar las características de la combinación actual
#     X_subset = X[list(combination)]

#     # Dividir los datos en conjuntos de entrenamiento y prueba
#     X_train, X_test, y_train, y_test = train_test_split(X_subset, y, test_size=0.2, random_state=123)

#     # Entrenar un modelo de regresión lineal utilizando solo las características seleccionadas
#     model = LinearRegression()
#     model.fit(X_train, y_train)

#     # Realizar predicciones en el conjunto de prueba
#     y_pred = model.predict(X_test)

#     # Evaluar el rendimiento del modelo utilizando el coeficiente de determinación (R^2)
#     r2 = r2_score(y_test, y_pred)

#     # Actualizar la mejor combinación de características si el rendimiento es mejor
#     if r2 > best_r2:
#         best_features = combination
#         best_r2 = r2

# print(f'Mejor combinación de características: {best_features}, R^2: {best_r2}')